Import нужных библиотек

In [3]:
import numpy as np
import pandas as pd

from tensorflow import keras as ks

Создание функция для токенизации, создания моделей, предсказания и сохранения результатов

In [29]:
def make_tokenizer(train_texts: pd.Series) -> ks.preprocessing.text.Tokenizer:
    tokenizer = ks.preprocessing.text.Tokenizer(num_words=1000)
    tokenizer.fit_on_texts(train_texts)
    return tokenizer


def make_lstm_model() -> ks.Model:
    model = ks.models.Sequential([
        ks.layers.Embedding(input_dim=1000, output_dim=64),
        ks.layers.LSTM(32),
        ks.layers.Dense(10, activation="relu"),
        ks.layers.Dense(1, activation="sigmoid"),
    ])
    model.compile(
        optimizer=ks.optimizers.Adam(),
        loss=ks.losses.BinaryCrossentropy(),
        metrics=[ks.metrics.BinaryAccuracy(threshold=0.5)]
    )
    return model

def make_dnn_model() -> ks.Model:
    model = ks.models.Sequential([
        ks.layers.Dense(100, activation="relu", input_dim=1000),
        ks.layers.Dense(10, activation="relu"),
        ks.layers.Dense(1, activation="sigmoid"),
    ])
    model.compile(
        optimizer=ks.optimizers.Adam(),
        loss=ks.losses.BinaryCrossentropy(),
        metrics=[ks.metrics.BinaryAccuracy(threshold=0.5)]
    )
    return model

def predict(model: ks.Model, data: np.ndarray):
    preds_proba = model.predict(data)
    preds = np.where(preds_proba > 0.5, 1, 0)
    preds = preds.squeeze()
    ans = pd.DataFrame({'Id': np.arange(0, preds.shape[0]), 'y': preds})
    ans.to_csv('submission.csv', index=False)

Чтение и preprocessing данных

In [5]:
train = pd.read_csv("/kaggle/input/simplesentiment/products_sentiment_train.tsv", sep='\t', header=None)
test = pd.read_csv("/kaggle/input/simplesentiment/products_sentiment_test.tsv", sep='\t')

In [6]:
X_raw_train = train[0]
Y_train = train[1].to_numpy()
X_raw_test = test["text"]

In [7]:
tokenizer = make_tokenizer(X_raw_train)
X_train = tokenizer.texts_to_matrix(X_raw_train)
X_test = tokenizer.texts_to_matrix(X_raw_test)

Тестирование LSTM модели. Она не обучается на этих данных.

In [12]:
lstm_model = make_lstm_model()

In [13]:
history = lstm_model.fit(X_train, Y_train, batch_size=16, epochs=5, validation_split=0.1)

Train on 1800 samples, validate on 200 samples
Epoch 1/5
1800/1800 [==============================] - 67s 37ms/sample - loss: 0.6686 - binary_accuracy: 0.6289 - val_loss: 0.6297 - val_binary_accuracy: 0.6900
Epoch 2/5
1800/1800 [==============================] - 63s 35ms/sample - loss: 0.6593 - binary_accuracy: 0.6311 - val_loss: 0.6261 - val_binary_accuracy: 0.6900
Epoch 3/5
1800/1800 [==============================] - 65s 36ms/sample - loss: 0.6600 - binary_accuracy: 0.6311 - val_loss: 0.6257 - val_binary_accuracy: 0.6900
Epoch 4/5
1800/1800 [==============================] - 64s 36ms/sample - loss: 0.6589 - binary_accuracy: 0.6311 - val_loss: 0.6245 - val_binary_accuracy: 0.6900
Epoch 5/5
1800/1800 [==============================] - 63s 35ms/sample - loss: 0.6585 - binary_accuracy: 0.6311 - val_loss: 0.6293 - val_binary_accuracy: 0.6900


Тестирование полносвязной сети. Она сильно переобучается на train, но дает ~0.75 на тесте

In [24]:
dnn_model = make_dnn_model()

In [26]:
history = dnn_model.fit(X_train, Y_train, batch_size=128, epochs=10, validation_split=0.1)

Train on 1800 samples, validate on 200 samples
Epoch 1/10
1800/1800 [==============================] - 0s 54us/sample - loss: 0.0045 - binary_accuracy: 0.9994 - val_loss: 0.9268 - val_binary_accuracy: 0.7300
Epoch 2/10
1800/1800 [==============================] - 0s 56us/sample - loss: 0.0044 - binary_accuracy: 0.9994 - val_loss: 0.9300 - val_binary_accuracy: 0.7350
Epoch 3/10
1800/1800 [==============================] - 0s 49us/sample - loss: 0.0042 - binary_accuracy: 0.9994 - val_loss: 0.9367 - val_binary_accuracy: 0.7300
Epoch 4/10
1800/1800 [==============================] - 0s 47us/sample - loss: 0.0041 - binary_accuracy: 0.9994 - val_loss: 0.9429 - val_binary_accuracy: 0.7300
Epoch 5/10
1800/1800 [==============================] - 0s 46us/sample - loss: 0.0039 - binary_accuracy: 1.0000 - val_loss: 0.9483 - val_binary_accuracy: 0.7300
Epoch 6/10
1800/1800 [==============================] - 0s 45us/sample - loss: 0.0038 - binary_accuracy: 1.0000 - val_loss: 0.9548 - val_binary_accu

Сохранение результатов предсказания для тестовых данных

In [28]:
predict(dnn_model, X_test)